In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os,time,pickle
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm

from joblib import dump, load
from utils import data_process, visualize
from utils.utils import train, test, summary, setup_seed
from utils.models import Att_CNN, CNN, TFEQ

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [3]:
# 读取数据
data,label=[],[]

# data.append(np.load('/home/Amin/EQ_Place/code/data/X_2_10.npy', allow_pickle=True))
# label.append(np.load('/home/Amin/EQ_Place/code/data/Y_2_10.npy'))
data.append(np.load('/home/Amin/EQ_Place/code/data/X_8_28.npy', allow_pickle=True))
label.append(np.load('/home/Amin/EQ_Place/code/data/Y_8_28.npy'))

data=np.vstack(data)
label=np.hstack(label)
print(pd.value_counts(label))
num=pd.value_counts(label)[1]

0    1118
1     372
dtype: int64


In [4]:
###======Settings for different test cases======###
SamplingRate = 100 # need to be changed, 25/50/100
Duration = 2 # need to be changed, 2/4/10
###============###
WindowSize = 2 * SamplingRate
original_SamplingRate = 100
rate = original_SamplingRate/SamplingRate

In [5]:
X_EQ, Y_EQ, Z_EQ = [], [], []
for i in tqdm(range(num)):
    X = data[i][0]
    X_peak = np.where(X == np.max(X))[0][0]
    start = X_peak - int(SamplingRate)
    end = X_peak + int(SamplingRate) * (Duration - 1)
    X_tg, Y_tg, Z_tg = data[i][0][start:end], data[i][1][start:end], data[i][
        2][start:end]
    X_tg = (X_tg - np.mean(X_tg))
    Y_tg = (Y_tg - np.mean(Y_tg)) 
    Z_tg = (Z_tg - np.mean(Z_tg)) 
    if len(X_tg) == SamplingRate * Duration:
        ## 2 sec sliding window with 1 sec overlap
        for j in np.arange(0, len(X_tg) - SamplingRate, SamplingRate):
            X_batch = X_tg[j:j + WindowSize]
            Y_batch = Y_tg[j:j + WindowSize]
            Z_batch = Z_tg[j:j + WindowSize]
            if len(X_batch) == WindowSize:
                X_EQ.append(X_batch)
                Y_EQ.append(Y_batch)
                Z_EQ.append(Z_batch)

X_EQ, Y_EQ, Z_EQ = np.asarray(X_EQ), np.asarray(Y_EQ), np.asarray(Z_EQ)

X_EQ = X_EQ.reshape(int(len(X_EQ) / (Duration - 1)), Duration - 1, WindowSize, 1)
Y_EQ = Y_EQ.reshape(int(len(Y_EQ) / (Duration - 1)), Duration - 1, WindowSize, 1)
Z_EQ = Z_EQ.reshape(int(len(Z_EQ) / (Duration - 1)), Duration - 1, WindowSize, 1)

100%|██████████| 372/372 [00:00<00:00, 6134.82it/s]


In [6]:
X_HA, Y_HA, Z_HA = [], [], []
for i in tqdm(range(num, len(data))):
    X, Y, Z = data[i][0], data[i][1], data[i][2]
    X_tg = (X - np.mean(X)) / 9.80665
    Y_tg = (Y - np.mean(Y)) / 9.80665
    Z_tg = (Z - np.mean(Z)) / 9.80665
    for j in np.arange(0, 400 - SamplingRate, SamplingRate):
        X_batch = X_tg[j:j + WindowSize]
        Y_batch = Y_tg[j:j + WindowSize]
        Z_batch = Z_tg[j:j + WindowSize]
        if len(X_batch) == WindowSize:
            X_HA.append(X_batch)
            Y_HA.append(Y_batch)
            Z_HA.append(Z_batch)

X_HA, Y_HA, Z_HA = np.asarray(X_HA), np.asarray(Y_HA), np.asarray(Z_HA)

X_HA = X_HA.reshape(X_HA.shape[0], X_HA.shape[1], 1)
Y_HA = Y_HA.reshape(Y_HA.shape[0], Y_HA.shape[1], 1)
Z_HA = Z_HA.reshape(Z_HA.shape[0], Z_HA.shape[1], 1)

100%|██████████| 1118/1118 [00:01<00:00, 610.23it/s]


In [12]:
indices = np.arange(len(X_EQ))
X_EQ_train, X_EQ_test, train_index, test_index = train_test_split(X_EQ, indices, test_size=0.995, random_state=42)
Y_EQ_train, Y_EQ_test = Y_EQ[train_index], Y_EQ[test_index]
Z_EQ_train, Z_EQ_test = Z_EQ[train_index], Z_EQ[test_index]

X_EQ_train = X_EQ_train.reshape(X_EQ_train.shape[0]*X_EQ_train.shape[1], X_EQ_train.shape[2], 1)
Y_EQ_train = Y_EQ_train.reshape(Y_EQ_train.shape[0]*Y_EQ_train.shape[1], Y_EQ_train.shape[2], 1)
Z_EQ_train = Z_EQ_train.reshape(Z_EQ_train.shape[0]*Z_EQ_train.shape[1], Z_EQ_train.shape[2], 1)

X_EQ_test = X_EQ_test.reshape(X_EQ_test.shape[0]*X_EQ_test.shape[1], X_EQ_test.shape[2], 1)
Y_EQ_test = Y_EQ_test.reshape(Y_EQ_test.shape[0]*Y_EQ_test.shape[1], Y_EQ_test.shape[2], 1)
Z_EQ_test = Z_EQ_test.reshape(Z_EQ_test.shape[0]*Z_EQ_test.shape[1], Z_EQ_test.shape[2], 1)

indices2 = np.arange(len(X_HA))
X_HA_train, X_HA_test, train_index2, test_index2 = train_test_split(X_HA, indices2, test_size=0.995, random_state=42)
Y_HA_train, Y_HA_test = Y_HA[train_index2], Y_HA[test_index2]
Z_HA_train, Z_HA_test = Z_HA[train_index2], Z_HA[test_index2]

In [13]:
X_EQ_train.shape

(1, 200, 1)

In [14]:
# EQ_X_train = np.dstack((X_EQ_train, Y_EQ_train, Z_EQ_train))
# HA_X_train = np.dstack((X_HA_train, Y_HA_train, Z_HA_train))

# EQ_y_train = np.ones(len(X_EQ_train))
# HA_y_train = np.zeros(len(X_HA_train))

# X_train = np.vstack((EQ_X_train, HA_X_train))
# y_train = np.hstack((EQ_y_train, HA_y_train)).reshape(-1,1)

EQ_X_test = np.dstack((X_EQ_test, Y_EQ_test, Z_EQ_test))
HA_X_test = np.dstack((X_HA_test, Y_HA_test, Z_HA_test))

EQ_y_test = np.ones(len(X_EQ_test))
HA_y_test = np.zeros(len(X_HA_test))

X_test = np.vstack((EQ_X_test, HA_X_test))
y_test = np.hstack((EQ_y_test, HA_y_test)).reshape(-1,1)


In [ ]:
dic1={'x_train':X_train,'y_train':y_train}

np.save("data_train.npy",dic1)

dic2={'x_test':X_test,'y_test':y_test}

np.save("data_test.npy",dic2)

In [8]:
x_train = torch.from_numpy(X_train).float()
y_train = torch.from_numpy(y_train).long()

x_test = torch.from_numpy(X_test).float()
y_test = torch.from_numpy(y_test).long()
traindata = TensorDataset(x_train, y_train)
testdata = TensorDataset(x_test, y_test)


train_loader = DataLoader(traindata, batch_size=32, shuffle=True)
test_loader = DataLoader(testdata, batch_size=32, shuffle=True)

In [9]:
%%time
setup_seed(42)
model = TFEQ(channel=3, time_in=200).cuda()
L = []
test_hist = []
optimizer = optim.Adam(model.parameters(), lr=0.001)
for epoch in range(1, 50 + 1):
    start = time.time()
    loss = train(model, train_loader, optimizer, epoch)
    L.append(loss)
    print("time {:.1f} sec:".format(time.time() - start))
    acc = test(model, test_loader)
    test_hist.append(acc)

Train Epoch: 1, Average Loss: 0.3439
time 0.8 sec:
Test set: Average loss: 0.3253, Accuracy: 1007/1119 (89.99%)

Train Epoch: 2, Average Loss: 0.3295
time 0.8 sec:
Test set: Average loss: 0.3258, Accuracy: 1007/1119 (89.99%)

Train Epoch: 3, Average Loss: 0.3245
time 0.8 sec:
Test set: Average loss: 0.3306, Accuracy: 1007/1119 (89.99%)

Train Epoch: 4, Average Loss: 0.3318
time 0.8 sec:
Test set: Average loss: 0.3252, Accuracy: 1007/1119 (89.99%)

Train Epoch: 5, Average Loss: 0.3294
time 0.8 sec:
Test set: Average loss: 0.3293, Accuracy: 1007/1119 (89.99%)

Train Epoch: 6, Average Loss: 0.3239
time 0.8 sec:
Test set: Average loss: 0.3350, Accuracy: 1007/1119 (89.99%)

Train Epoch: 7, Average Loss: 0.3269
time 0.8 sec:
Test set: Average loss: 0.3345, Accuracy: 1007/1119 (89.99%)

Train Epoch: 8, Average Loss: 0.3261
time 0.8 sec:
Test set: Average loss: 0.3252, Accuracy: 1007/1119 (89.99%)

Train Epoch: 9, Average Loss: 0.3235
time 0.8 sec:
Test set: Average loss: 0.3268, Accuracy: 100

In [10]:
model.eval()
y_pred = []
y_test = []
with torch.no_grad():
    for data, target in test_loader:
        data = data.cuda()
        output = model(data)
        pred = output.max(1, keepdim=True)[1]  
        pred = pred.cpu().numpy().squeeze()
        y_pred.append(pred)
        y_test.append(target.numpy().squeeze())
y_pred=np.hstack(y_pred)
y_test=np.hstack(y_test)

summary(y_pred,y_test)

confusion matrix:
 [[1007    3]
 [   0  109]]
Roc score: 0.9985
F1 score: 0.9864
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1010
           1       0.97      1.00      0.99       109

    accuracy                           1.00      1119
   macro avg       0.99      1.00      0.99      1119
weighted avg       1.00      1.00      1.00      1119

